# Build model V1

In [ ]:
# Net Data
# Testing
# batch=1
# subdivisions=1
# Training
# batch=64
# subdivisions=16
# width=608
# height=608
# channels=3
# momentum=0.9
# decay=0.0005
# angle=0
# saturation = 1.5
# exposure = 1.5
# hue=.1

# learning_rate=0.001
# burn_in=1000
# max_batches = 500200
# policy=steps
# steps=400000,450000
# scales=.1,.1

# Conv
# Layer 1

lay1_1 = Conv2D(32, 3, strides=(1, 1), padding='same')(input_image)
lay1_2 = BatchNormalization(epsilon=0.001)(lay1_1)
lay1_3 = LeakyReLU(alpha=0.1)(lay1_2)

# Downsample

# Conv
# Layer 2
lay2_1 = ZeroPadding2D(((1,0),(1,0)))(lay1_3)
lay2_2 = Conv2D(64, 3, strides=(2, 2), padding='valid')(lay2_1)
lay2_3 = BatchNormalization(epsilon=0.001)(lay2_2)
lay2_4 = LeakyReLU(alpha=0.1)(lay2_3)

# Conv
# Layer 3

lay3_1 = Conv2D(32, 1, strides=(1, 1), padding='same')(lay2_4)
lay3_2 = BatchNormalization(epsilon=0.001)(lay3_1)
lay3_3 = LeakyReLU(alpha=0.1)(lay3_2)

# Conv
# Layer 4

lay4_1 = Conv2D(64, 3, strides=(1, 1), padding='same')(lay3_3)
lay4_2 = BatchNormalization(epsilon=0.001)(lay4_1)
lay4_3 = LeakyReLU(alpha=0.1)(lay4_2)

# Shortcut -> Conv
# Layer 5

lay5_1 = Conv2D(32, 1, strides=(1, 1), padding='same')(lay4_3+lay2_4)
lay5_2 = Linear(lay5_1)

# Downsample

# Conv
# Layer 6

lay6_1 = ZeroPadding2D(((1,0),(1,0)))(lay5_2)
lay6_2 = Conv2D(128, 3, strides=(2, 2), padding='valid')(lay6_1)
lay6_3 = BatchNormalization(epsilon=0.001)(lay6_2)
lay6_4 = LeakyReLU(alpha=0.1)(lay6_3)

# Conv
# Layer 7

lay7_1 = Conv2D(64, 1, strides=(1, 1), padding='same')(lay6_4)
lay7_2 = BatchNormalization(epsilon=0.001)(lay7_1)
lay7_3 = LeakyReLU(alpha=0.1)(lay7_2)

# Conv
# Layer 8

lay8_1 = Conv2D(128, 3, strides=(1, 1), padding='same')(lay7_3)
lay8_2 = BatchNormalization(epsilon=0.001)(lay8_1)
lay8_3 = LeakyReLU(alpha=0.1)(lay8_2)

# Shortcut -> Conv
# Layer 9

lay9_1 = Conv2D(64, 1, strides=(1, 1), padding='same')(lay8_3+lay6_3)
lay9_2 = Linear(lay9_1)

# Conv
# Layer 10

lay10_1 = Conv2D(64, 1, strides=(1, 1), padding='same')(lay9_2)
lay10_2 = BatchNormalization(epsilon=0.001)(lay10_1)
lay10_3 = LeakyReLU(alpha=0.1)(lay10_2)

# Conv
# Layer 11

lay11_1 = Conv2D(128, 3, strides=(1, 1), padding='same')(lay10_3)
lay11_2 = BatchNormalization(epsilon=0.001)(lay11_1)
lay11_3 = LeakyReLU(alpha=0.1)(lay11_2)

# Shortcut -> Conv
# layer 12

lay12_1 = Conv2D(64, 1, strides=(1, 1), padding='same')(lay11_3+lay9_3)
lay12_2 = Linear(lay9_1)

# Downsample

# Conv 
# Layer 13

lay13_1 = ZeroPadding2D(((1,0),(1,0)))(lay12_2)
lay13_2 = Conv2D(256, 3, strides=(2, 2), padding='valid')(lay13_1)
lay13_3 = BatchNormalization(epsilon=0.001)(lay13_2)
lay13_4 = LeakyReLU(alpha=0.1)(lay13_3)

# Conv
# Layer 14

lay14_1 = Conv2D(128, 1, strides=(1, 1), padding='same')(lay13_4)
lay14_2 = BatchNormalization(epsilon=0.001)(lay14_1)
lay14_3 = LeakyReLU(alpha=0.1)(lay14_2)

# Conv
# Layer 15

lay15_1 = Conv2D(256, 3, strides=(1, 1), padding='same')(lay14_3)
lay15_2 = BatchNormalization(epsilon=0.001)(lay15_1)
lay15_3 = LeakyReLU(alpha=0.1)(lay15_2)


# Shortcut -> Conv
from=-3
activation=linear

# Conv
# Layer
batch_normalize=1
filters=128
size=1
stride=1
pad=1
activation=leaky

# Conv
# Layer
batch_normalize=1
filters=256
size=3
stride=1
pad=1
activation=leaky

# Shortcut -> Conv
from=-3
activation=linear

# Conv
# Layer
batch_normalize=1
filters=128
size=1
stride=1
pad=1
activation=leaky

# Conv
# Layer
batch_normalize=1
filters=256
size=3
stride=1
pad=1
activation=leaky

# Shortcut -> Conv
from=-3
activation=linear

# Conv
# Layer
batch_normalize=1
filters=128
size=1
stride=1
pad=1
activation=leaky

# Conv
batch_normalize=1
filters=256
size=3
stride=1
pad=1
activation=leaky

# Shortcut -> Conv
from=-3
activation=linear


# Conv
batch_normalize=1
filters=128
size=1
stride=1
pad=1
activation=leaky

# Conv
batch_normalize=1
filters=256
size=3
stride=1
pad=1
activation=leaky

# Shortcut -> Conv
from=-3
activation=linear

# Conv
batch_normalize=1
filters=128
size=1
stride=1
pad=1
activation=leaky

# Conv
batch_normalize=1
filters=256
size=3
stride=1
pad=1
activation=leaky

# Shortcut -> Conv
from=-3
activation=linear

# Conv
batch_normalize=1
filters=128
size=1
stride=1
pad=1
activation=leaky

# Conv
batch_normalize=1
filters=256
size=3
stride=1
pad=1
activation=leaky

# Shortcut -> Conv
from=-3
activation=linear

# Conv
batch_normalize=1
filters=128
size=1
stride=1
pad=1
activation=leaky

# Conv
batch_normalize=1
filters=256
size=3
stride=1
pad=1
activation=leaky

# Shortcut -> Conv
from=-3
activation=linear

# Downsample

# Conv
batch_normalize=1
filters=512
size=3
stride=2
pad=1
activation=leaky

# Conv
batch_normalize=1
filters=256
size=1
stride=1
pad=1
activation=leaky

# Conv
batch_normalize=1
filters=512
size=3
stride=1
pad=1
activation=leaky

# Shortcut -> Conv
from=-3
activation=linear


# Conv
batch_normalize=1
filters=256
size=1
stride=1
pad=1
activation=leaky

# Conv
batch_normalize=1
filters=512
size=3
stride=1
pad=1
activation=leaky

# Shortcut -> Conv
from=-3
activation=linear


# Conv
batch_normalize=1
filters=256
size=1
stride=1
pad=1
activation=leaky

# Conv
batch_normalize=1
filters=512
size=3
stride=1
pad=1
activation=leaky

# Shortcut -> Conv
from=-3
activation=linear


# Conv
batch_normalize=1
filters=256
size=1
stride=1
pad=1
activation=leaky

# Conv
batch_normalize=1
filters=512
size=3
stride=1
pad=1
activation=leaky

# Shortcut -> Conv
from=-3
activation=linear

# Conv
batch_normalize=1
filters=256
size=1
stride=1
pad=1
activation=leaky

# Conv
batch_normalize=1
filters=512
size=3
stride=1
pad=1
activation=leaky

# Shortcut -> Conv
from=-3
activation=linear


# Conv
batch_normalize=1
filters=256
size=1
stride=1
pad=1
activation=leaky

# Conv
batch_normalize=1
filters=512
size=3
stride=1
pad=1
activation=leaky

# Shortcut -> Conv
from=-3
activation=linear


# Conv
batch_normalize=1
filters=256
size=1
stride=1
pad=1
activation=leaky

# Conv
batch_normalize=1
filters=512
size=3
stride=1
pad=1
activation=leaky

# Shortcut -> Conv
from=-3
activation=linear

# Conv
batch_normalize=1
filters=256
size=1
stride=1
pad=1
activation=leaky

# Conv
batch_normalize=1
filters=512
size=3
stride=1
pad=1
activation=leaky

# Shortcut -> Conv
from=-3
activation=linear

# Downsample

# Conv
batch_normalize=1
filters=1024
size=3
stride=2
pad=1
activation=leaky

# Conv
batch_normalize=1
filters=512
size=1
stride=1
pad=1
activation=leaky

# Conv
batch_normalize=1
filters=1024
size=3
stride=1
pad=1
activation=leaky

# Shortcut -> Conv
from=-3
activation=linear

# Conv
batch_normalize=1
filters=512
size=1
stride=1
pad=1
activation=leaky

# Conv
batch_normalize=1
filters=1024
size=3
stride=1
pad=1
activation=leaky

# Shortcut -> Conv
from=-3
activation=linear

# Conv
batch_normalize=1
filters=512
size=1
stride=1
pad=1
activation=leaky

# Conv
batch_normalize=1
filters=1024
size=3
stride=1
pad=1
activation=leaky

# Shortcut -> Conv
from=-3
activation=linear

# Conv
batch_normalize=1
filters=512
size=1
stride=1
pad=1
activation=leaky

# Conv
batch_normalize=1
filters=1024
size=3
stride=1
pad=1
activation=leaky

# Shortcut -> Conv
from=-3
activation=linear

######################

# Conv
batch_normalize=1
filters=512
size=1
stride=1
pad=1
activation=leaky

# Conv
batch_normalize=1
size=3
stride=1
pad=1
filters=1024
activation=leaky

# Conv
batch_normalize=1
filters=512
size=1
stride=1
pad=1
activation=leaky

# Conv
batch_normalize=1
size=3
stride=1
pad=1
filters=1024
activation=leaky

# Conv
batch_normalize=1
filters=512
size=1
stride=1
pad=1
activation=leaky

# Conv
batch_normalize=1
size=3
stride=1
pad=1
filters=1024
activation=leaky

# Conv
size=1
stride=1
pad=1
filters=255
activation=linear


[yolo]
mask = 6,7,8
anchors = 10,13,  16,30,  33,23,  30,61,  62,45,  59,119,  116,90,  156,198,  373,326
classes=80
num=9
jitter=.3
ignore_thresh = .7
truth_thresh = 1
random=1


[route]
layers = -4

# Conv
batch_normalize=1
filters=256
size=1
stride=1
pad=1
activation=leaky

[upsample]
stride=2

[route]
layers = -1, 61



# Conv
batch_normalize=1
filters=256
size=1
stride=1
pad=1
activation=leaky

# Conv
batch_normalize=1
size=3
stride=1
pad=1
filters=512
activation=leaky

# Conv
batch_normalize=1
filters=256
size=1
stride=1
pad=1
activation=leaky

# Conv
batch_normalize=1
size=3
stride=1
pad=1
filters=512
activation=leaky

# Conv
batch_normalize=1
filters=256
size=1
stride=1
pad=1
activation=leaky

# Conv
batch_normalize=1
size=3
stride=1
pad=1
filters=512
activation=leaky

# Conv
size=1
stride=1
pad=1
filters=255
activation=linear


[yolo]
mask = 3,4,5
anchors = 10,13,  16,30,  33,23,  30,61,  62,45,  59,119,  116,90,  156,198,  373,326
classes=80
num=9
jitter=.3
ignore_thresh = .7
truth_thresh = 1
random=1



[route]
layers = -4

# Conv
batch_normalize=1
filters=128
size=1
stride=1
pad=1
activation=leaky

[upsample]
stride=2

[route]
layers = -1, 36



# Conv
batch_normalize=1
filters=128
size=1
stride=1
pad=1
activation=leaky

# Conv
batch_normalize=1
size=3
stride=1
pad=1
filters=256
activation=leaky

# Conv
batch_normalize=1
filters=128
size=1
stride=1
pad=1
activation=leaky

# Conv
batch_normalize=1
size=3
stride=1
pad=1
filters=256
activation=leaky

# Conv
batch_normalize=1
filters=128
size=1
stride=1
pad=1
activation=leaky

# Conv
batch_normalize=1
size=3
stride=1
pad=1
filters=256
activation=leaky

# Conv
size=1
stride=1
pad=1
filters=255
activation=linear


[yolo]
mask = 0,1,2
anchors = 10,13,  16,30,  33,23,  30,61,  62,45,  59,119,  116,90,  156,198,  373,326
classes=80
num=9
jitter=.3
ignore_thresh = .7
truth_thresh = 1
random=1

